In [12]:
%pip install --upgrade openai python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [13]:
# Configure logging and load environment variables from the nearest .env
# - Uses a namespaced logger to avoid duplicate handlers in notebooks
# - Searches parent directories for a .env so you can run from anywhere

import os
import sys
import pathlib
import logging
from dotenv import load_dotenv

# Create a logger suitable for notebooks and larger projects
logger = logging.getLogger("openai_api_demo")
if not logger.handlers:
    stream_handler = logging.StreamHandler(sys.stdout)
    stream_handler.setFormatter(logging.Formatter("[%(levelname)s] %(message)s"))
    logger.addHandler(stream_handler)
logger.setLevel(logging.INFO)

# Find the nearest .env file by walking up the directory tree
current_working_dir = pathlib.Path.cwd()
found_env_path = None
for parent_dir in [current_working_dir] + list(current_working_dir.parents):
    candidate_env = parent_dir / ".env"
    if candidate_env.is_file():
        found_env_path = candidate_env
        break

if found_env_path is None:
    logger.warning("No .env file found – make sure OPENAI_API_KEY is set in your environment.")
else:
    logger.info("🔑 Loading environment from %s", found_env_path)
    load_dotenv(found_env_path)



[INFO] 🔑 Loading environment from /home/superdev/projects/OpenMates/.env


In [14]:
# Initialize the OpenAI client
# Required env var: OPENAI_API_KEY
# Optional: OPENAI_BASE_URL, OPENAI_ORG_ID, OPENAI_PROJECT_ID

import os
from openai import OpenAI

openai_api_key = os.getenv("SECRET__OPENAI__API_KEY")
if not openai_api_key:
    logger.error("OPENAI_API_KEY is not set. Please define it in your environment or .env file.")

base_url = os.getenv("OPENAI_BASE_URL")  # e.g., https://api.openai.com/v1 (default)
organization = os.getenv("OPENAI_ORG_ID")
project = os.getenv("OPENAI_PROJECT_ID")

client = OpenAI(
    api_key=openai_api_key,
    base_url=base_url if base_url else None,
    organization=organization if organization else None,
    project=project if project else None,
)

logger.info("OpenAI client initialized%s%s%s.",
            f" with base_url={base_url}" if base_url else "",
            f", org={organization}" if organization else "",
            f", project={project}" if project else "")


[INFO] OpenAI client initialized.


In [15]:
# Make a simple Chat Completions API call and log the response
# - Model can be set via env var OPENAI_CHAT_MODEL, defaults to 'gpt-5'
# - Logs concise info plus full JSON at debug level for troubleshooting

chat_model = os.getenv("OPENAI_CHAT_MODEL", "gpt-5")

# Minimal prompt to validate end-to-end connectivity
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Capital of France?"},
]

try:
    response = client.chat.completions.create(
        model=chat_model,
        messages=messages
        # Note: temperature is not supported for gpt-5
    )
    logger.info("Chat completion received from model='%s'.", chat_model)
    # The SDK exposes .to_json() for structured logging
    logger.debug("Full response JSON: %s", response.to_json())

    model_reply = response.choices[0].message.content if response and response.choices else ""
    logger.info("Model reply: %s", model_reply)
except Exception as exc:
    logger.exception("Failed to create chat completion: %s", exc)


[INFO] Chat completion received from model='gpt-5'.
[INFO] Model reply: Paris.


In [ ]:
# Make a simple Chat Completions API call and log the response
# - Model can be set via env var OPENAI_CHAT_MODEL, defaults to 'gpt-5'
# - Logs concise info plus full JSON at debug level for troubleshooting

chat_model = os.getenv("OPENAI_CHAT_MODEL", "gpt-5")

# Minimal prompt to validate end-to-end connectivity
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "how do I integrate Oxylabs proxy into my python code? and can I integrate it also with the youtube_transcript_api pip package?"},
]

try:
    response = client.chat.completions.create(
        model=chat_model,
        messages=messages
        # Note: temperature is not supported for gpt-5
    )
    logger.info("Chat completion received from model='%s'.", chat_model)
    # The SDK exposes .to_json() for structured logging
    logger.debug("Full response JSON: %s", response.to_json())

    model_reply = response.choices[0].message.content if response and response.choices else ""
    logger.info("Model reply: %s", model_reply)
except Exception as exc:
    logger.exception("Failed to create chat completion: %s", exc)


[INFO] Chat completion received from model='gpt-5'.
[INFO] Model reply: Here are the most common ways to use Oxylabs proxies in Python, plus how to use them with youtube-transcript-api.

1) Using Oxylabs with requests
- Build a proxy URL with your Oxylabs credentials and endpoint (check your Oxylabs dashboard for the correct host/port and username format for your zone/type).

Example:
import requests

username = "customer-<your_customer>-zone-<your_zone>"  # see Oxylabs dashboard for your exact format
password = "<your_password>"
endpoint = "pr.oxylabs.io"
port = 7777

proxy = f"http://{username}:{password}@{endpoint}:{port}"
proxies = {
    "http": proxy,
    "https": proxy,  # yes, still "http://" scheme here; requests will use CONNECT for HTTPS
}

r = requests.get("https://httpbin.org/ip", proxies=proxies, timeout=30)
print(r.text)

Notes:
- For country targeting or sticky sessions, Oxylabs lets you add options into the username (e.g., country or session). Use the exact pattern show